In [1]:
import mne

import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import MinMaxScaler
from IPython.display import clear_output
import os

from scipy import io
from scipy.stats import entropy

import pywt
import eegraph

from utils import *

data_path = 'AMIGOS/preprocessed/'

In [2]:
df = build_dataframe(data_path)

In [3]:
df0 = df.copy()

In [4]:
### create different types of scaled eeg data based on filtration borders
df.EEG = df0.EEG.apply(lambda x: scale_eeg(x, 4, 45))

with open(data_path + 'pd_model_dataset_ica_4_45.pkl', 'wb') as f:
    pickle.dump(df, f)

In [5]:
### make discrete wavelet transform and save new files

for name in ['pd_model_dataset_ica_4_45']:
    with open(data_path + name + '.pkl', 'rb') as f:
        df = pickle.load(f)
     
    for dwt_method in ['energy', 'entropy']:
        data_list = [discrete_wavelet_transform(eeg, dwt_method) for eeg in df.EEG]
        data_list = zero_padding(data_list, 80) 
        df['dwt'] = data_list

        with open(data_path + name + '_dwt_' + dwt_method + '.pkl', 'wb') as f:
            pickle.dump(df, f)
            
    os.remove(data_path + name + '.pkl')

In [6]:
def save_edf_eegs(data_path, EEGS):
    for i in range(len(EEGS)):
        eegmat = EEGS[i]
        info = mne.create_info(['AF3', 'F7', 'F3', 'FC5', 'T7', 
                                    'P7', 'O1', 'O2', 'P8', 'T8', 
                                    'FC6', 'F4', 'F8', 'AF4'], sfreq=128, 
                                  ch_types= 14*['eeg'])

        raw = mne.io.RawArray(eegmat, info)
        raw.set_montage('standard_1020')

        mne.export.export_raw(data_path + 'eegraph_files/eeg_{}.edf'.format(i), raw, fmt='edf')

## Functional features

##### Go to final_create_eds_files.ipynb and run cells for all pickle file names
##### Continue after all

In [16]:
for name in [x for x in os.listdir(data_path) if '_dwt_' in x and '.pkl' in x and '_func_data_' not in x]:
    with open(data_path + name, 'rb') as f:
        df = pickle.load(f)
    
    edfs_dict = {}

    for i in range(df.shape[0]):
        G = eegraph.Graph()
        G.load_data(data_path + 'eegraph_files_' + name.split('.')[0] +  '/eeg_{}.edf'.format(i))
        edfs_dict[i] = G
        clear_output(wait=False)
    
    for window_size in [2,3]:
        matrices = []
        for i in edfs_dict.keys():
            print(i)
            graphs, connectivity_matrix = edfs_dict[i].modelate(window_size = window_size, 
                                                                connectivity = 'imag_coherence', 
                                                                bands = ['delta','theta','alpha','beta','gamma'])
            matrices.append(connectivity_matrix)
            clear_output(wait=False)
            
        df['im_coh_'+ str(window_size)] = matrices
            
        matrices = []
        for i in edfs_dict.keys():
            print(i)
            graphs, connectivity_matrix = edfs_dict[i].modelate(window_size = window_size, 
                                                                connectivity = 'pearson_correlation')
            matrices.append(connectivity_matrix)
            clear_output(wait=False)
            
        df['pearson_corr_'+ str(window_size)] = matrices
        
    with open(data_path + name.replace('.pkl', '') + '_func_data.pkl', 'wb') as f:
        pickle.dump(df, f)
            
    os.remove(data_path + name)

## Spectral features
##### Go to final_encoder_features.ipynb and create autoencoder features
